In [1]:
import os

import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

from dataset import CatsAndDogs

NameError: name 'default_loader' is not defined

In [1]:
dataset_path = "/home/luca/datasets/cats-and-dogs-breeds-classification-oxford-dataset"

images_path = os.path.join(dataset_path, "images", "images")
annotation_path = os.path.join(dataset_path, "annotations", "annotations")

NameError: name 'os' is not defined

In [12]:
def make_dataset(images_path, annotation_path):
    samples = []
    with open(annotation_path, "r") as file:
        for line in file.readlines():
            image, class_idx, species, breed = line.rsplit()

            item = (os.path.join(images_path, "{}.jpg".format(image)), int(class_idx))
            samples.append(item)
            
    return samples

class CatsAndDogs(Dataset):

    def __init__(self, dataset_path, mode, loader=default_loader, transform=None, target_transform=None):
        """
        Parameters
        ----------
        dataset_path: str,
            path to the folder containing the frames in .jpg format.
        annotation_path: str,
            path to the folder containing the annotation .txt files.
        set_type: str,
            choose between trainval, test.
        loader: Pytorch loader,
            loader for the images. Defaults to the default loader (PIL).
        transform: torch.transform,
            transformation to apply to the samples.
        target_transform: torch.transform,
            transformation to apply to the targets (i.e. labels).
        """

        self.dataset_path = dataset_path
        self.mode = mode

        self.images_path = os.path.join(dataset_path, "images", "images")
        self.annotation_path = os.path.join(dataset_path, "annotations", "annotations", "{}.txt".format(self.mode))
        

        self.transform = transform
        self.target_transform = target_transform

        samples = make_dataset(self.images_path, self.annotation_path)
        
        self.samples = samples
        self.targets = [s[1] for s in samples]

        self.loader = loader
        self.classes = [i for i in range(1, 38)]
    

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):

        sample, target = self.samples[index]
        sample = self.loader(sample)

        if self.transform is not None:
            sample = self.transform(sample)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

In [28]:
batch_size = 32 
num_workers = 12

train_transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

train_dataset = CatsAndDogs(dataset_path, mode="trainval", transform=train_transform)
test_dataset = CatsAndDogs(dataset_path, mode="test", transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
train_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [29]:
for im, tar in train_loader:
    print(im.shape, tar)

torch.Size([32, 3, 224, 224]) tensor([31,  6, 14, 12, 30,  9, 14,  7, 21, 19, 35, 31,  9, 27, 16, 12, 29, 33,
        22,  8, 34, 30, 16, 30, 20, 12,  7, 29, 34, 32, 31, 27])
torch.Size([32, 3, 224, 224]) tensor([30, 26, 36, 20, 25, 22, 20, 21, 13,  7,  4,  6, 17, 26,  4, 19,  1, 35,
        35, 29, 25, 31, 31, 23, 16,  9, 34, 36, 22, 16,  3, 24])
torch.Size([32, 3, 224, 224]) tensor([ 5, 16, 16,  5, 24,  4,  6,  6, 36, 34, 19, 36, 23, 29, 30, 26, 28, 16,
         3, 21, 14, 14, 19, 23, 34, 20, 22, 35,  7, 13, 17, 36])
torch.Size([32, 3, 224, 224]) tensor([35, 33, 33, 12, 32, 18, 33, 12,  8,  9, 13, 16,  1, 14,  6, 22,  7, 17,
        25, 32, 36, 18, 10, 29, 20,  4,  5, 25,  4, 17, 14, 25])
torch.Size([32, 3, 224, 224]) tensor([ 3,  7, 32, 32, 17, 21, 29, 16, 19, 32,  8, 12, 28, 29, 33, 10, 32,  9,
        18, 30, 12,  7,  9, 25, 31,  2, 17, 30, 37, 34, 13, 15])
torch.Size([32, 3, 224, 224]) tensor([20, 15, 37, 19, 10, 13, 14, 17, 11,  9, 23, 36,  4, 19, 36,  4,  4,  1,
        20, 32,

torch.Size([32, 3, 224, 224]) tensor([27, 14, 37, 17, 33, 21, 25, 31,  5, 17, 13, 33, 30, 33, 26, 29, 32, 18,
        11, 22, 19, 20, 27, 27, 10, 15, 35, 21, 29,  8,  3, 22])
torch.Size([32, 3, 224, 224]) tensor([22, 19, 33, 33, 27, 22,  7, 21, 34, 17,  3, 17, 23, 37,  2, 19,  6, 15,
        13, 34, 12, 34, 12, 30, 34, 11,  9,  6, 34, 34, 11, 16])
torch.Size([32, 3, 224, 224]) tensor([ 4, 28,  9, 15, 20, 26,  3, 19,  1, 18,  5, 30, 22, 31, 21, 34, 24, 19,
        21,  2,  8, 18, 24, 31, 36,  9, 15, 34,  6,  8, 26, 37])
torch.Size([32, 3, 224, 224]) tensor([20, 29, 19, 18, 19, 14, 24, 15, 26, 14, 35, 17, 15, 29, 26, 21, 36, 18,
        33,  6, 30, 34, 27,  7,  1, 10,  6, 26, 34, 18, 12, 27])
torch.Size([32, 3, 224, 224]) tensor([ 9, 30,  2,  6, 17, 25,  9, 34, 22, 30,  6, 19,  8,  9, 10, 10, 10, 11,
        27,  8,  3, 10,  2, 26, 37, 24, 20, 31, 15, 25, 12,  9])
torch.Size([32, 3, 224, 224]) tensor([ 5, 30, 11, 26,  8, 37, 25, 19, 36, 21, 29, 15, 31, 13, 26,  1, 32, 36,
        28, 31,

torch.Size([32, 3, 224, 224]) tensor([26,  6, 30, 30,  1, 31, 35, 24,  7,  6, 36, 34, 35,  4, 17, 12, 36, 32,
         6, 13, 29, 22,  5, 24, 13,  7, 23, 17, 18, 30, 16,  1])
torch.Size([32, 3, 224, 224]) tensor([18,  3, 13,  7, 17,  1,  9, 23, 26, 18,  7, 29, 13, 21, 15, 23,  2,  5,
        22, 24,  1, 26, 21, 14, 11, 24,  3, 32, 14, 30,  8,  4])
torch.Size([32, 3, 224, 224]) tensor([29, 21, 21, 16,  9, 21, 37, 13, 14, 26, 20,  5,  4, 22, 11,  6, 28, 32,
         2, 29, 27, 14, 32, 12, 25, 33, 27,  3,  7, 14, 12, 19])
torch.Size([32, 3, 224, 224]) tensor([22, 30, 30, 34,  9,  8, 31, 32,  1, 14,  3, 26, 15, 25, 13,  9, 36, 26,
        28, 11, 12, 17, 32, 23,  9, 22, 30,  3, 12, 35, 10, 29])
torch.Size([32, 3, 224, 224]) tensor([34, 21, 20, 18, 29, 25, 13, 17, 30,  6, 10, 35, 19, 32, 21, 34, 19, 13,
        16, 34, 21, 33, 14, 22,  2, 18,  8,  7, 21, 29, 35, 31])
torch.Size([32, 3, 224, 224]) tensor([28, 11, 30,  5, 31, 21, 11, 28,  6, 14, 15, 19, 13, 18, 11,  1,  1,  9,
         7, 11,